# OCR

> OCR evaluation reports

In [ ]:
#| default_exp ocr

In [ ]:
#| export
import os
import re
import time
from io import BytesIO
from dotenv import load_dotenv
import base64
from fastcore.all import *
from rich import print
import urllib.parse
from pathlib import Path
from tqdm import tqdm

import pandas as pd
from mistralai import Mistral
from PIL import Image
# from tqdm import tqdm

from evaluatr.readers import load_evals


In [ ]:
#| export
load_dotenv()
mistral_api_key = os.getenv("MISTRAL_API_KEY")

In [ ]:
#| exports
def get_doc_subtype(
    id:str, # ID of the evaluation
    fname:str, # Name of the file
    evals # Evaluations data
    )->str: # Document Subtype
    "Get Document Subtype for a given file in the evaluation dataset"
    eval_data = L(evals).filter(lambda x: x['id']==id)
    if not eval_data: return None
    
    docs = L(eval_data[0]['docs'])
    matches = docs.filter(lambda x: Path(x['File URL']).name==fname)
    return matches[0]['Document Subtype'] if matches else None

Given an evaluation `id` and a `pdf` file name of one of its supporting doc we can check its `subtype`.

For instance, for the "Final Evaluation of the EU-IOM Joint ..." evaluation:

In [ ]:
#| eval: false
fname_json = '../_data/output/evaluations.json'
path_pdf = Path('../_data/pdf_library')

title = 'Final Evaluation of the EU-IOM Joint Initiative for migrant protection and reintegration in the Horn of Africa'
evals = load_evals(fname_json)

results = [o for o in evals.filter(lambda x: title.lower() in x['meta']['Title'].lower())]; results

[{'id': '49d2fba781b6a7c0d94577479636ee6f',
  'docs': [{'Document Subtype': 'Evaluation report',
    'File URL': 'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Abridged%20Evaluation%20Report_%20Final_Olta%20NDOJA.pdf',
    'File description': 'Evaluation Report'},
   {'Document Subtype': 'Evaluation brief',
    'File URL': 'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Evaluation%20Learning%20Brief_Final_Olta%20NDOJA.pdf',
    'File description': 'Evaluation Brief'},
   {'Document Subtype': 'Annexes',
    'File URL': 'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Final%20Evaluation%20Report%20Final_Olta%20NDOJA.pdf',
    'File description': 'Abridged Report'},
   {'Document Subtype': 'Management response',
    'File URL': 'https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/HoA%20EU%20JI%20Final%20Eval%20-%20Management%20Response%20Matrix%20-%20Final.pdf',
    'File description': 'Manageme

In [ ]:
#| eval: false
id = '49d2fba781b6a7c0d94577479636ee6f'
for o in path_pdf.ls().filter(lambda x: x.name == id)[0].ls():
    print(f'Name: {o.name}\nSubtype: {get_doc_subtype(id, o.name, evals)}')

Name: Evaluation%20Learning%20Brief_Final_Olta%20NDOJA.pdf
Subtype: Evaluation brief

Name: HoA%20EU%20JI%20Final%20Eval%20-%20Management%20Response%20Matrix%20-%20Final.pdf
Subtype: Management response

Name: Abridged%20Evaluation%20Report_%20Final_Olta%20NDOJA.pdf
Subtype: Evaluation report

Name: Final%20Evaluation%20Report%20Final_Olta%20NDOJA.pdf
Subtype: Annexes

In [ ]:
#| export
src_dir = Path("../_data/")

## Mistral client

In [ ]:
#| export
def ocr(
    pdf_path:Union[Path,str], # Path to the PDF file to process
    model:str="mistral-ocr-latest", # Model name to use for OCR processing
    include_images:bool=True, # Whether to include base64-encoded images in the response
    api_key:str=mistral_api_key # Mistral API key for authentication
):
    "Upload a PDF file and process it with OCR in one go"
    if isinstance(pdf_path, str): pdf_path = Path(pdf_path)
    cli = Mistral(api_key=api_key)
    
    uploaded_pdf = cli.files.upload(
        file={
            "file_name": pdf_path.stem,
            "content": pdf_path.read_bytes(),
        },
        purpose="ocr"
    )

    signed_url = cli.files.get_signed_url(file_id=uploaded_pdf.id)
    
    return cli.ocr.process(
        model=model,
        document={
            "type": "document_url",
            "document_url": signed_url.url,
        },
        include_image_base64=include_images
    )

In [ ]:
# pdf_report = '../_data/pdf_library/49d2fba781b6a7c0d94577479636ee6f/Abridged%20Evaluation%20Report_%20Final_Olta%20NDOJA.pdf'

In [ ]:
# Note: filename will be cleaned upstream in a next version
#| exports
def clean_pdf_name(pdf_name: str) -> str:
    """
    Clean PDF name to create folder-friendly string.
    Removes special characters and spaces, replaces with underscores.
    """
    # Remove URL encoding
    import urllib.parse
    pdf_name = urllib.parse.unquote(pdf_name)
    
    # Replace spaces and special characters with underscores
    # Replace any character that is not a word character (\w), whitespace (\s), or hyphen (-) with underscore
    cleaned = re.sub(r'[^\w\s-]', '_', pdf_name)
    
    # Replace any sequence of hyphens or whitespace with a single underscore
    cleaned = re.sub(r'[-\s]+', '_', cleaned)
    
    # Replace multiple consecutive underscores with a single underscore
    cleaned = re.sub(r'_+', '_', cleaned)
    cleaned = cleaned.strip('_')  # Remove leading/trailing underscores
    
    return cleaned.lower()

In [ ]:
clean_pdf_name("Final%20Evaluation%20Report%20Final_Olta%20NDOJA.pdf")

'final_evaluation_report_final_olta_ndoja_pdf'

In [ ]:
# Notes, brainstorm
## we can fix the headings hierarchy passing the headings to an LLM
## we can also pass the toc if found
## we could then replace each heading with the corresponding fixed text + the page number
## Hence we could ask an LLM to propose semantic chunks based on titles, level and numbe or pages

In [ ]:
# pdf_report

In [ ]:
#| eval: false
r = ocr(pdf_report)

In [ ]:
#| eval: false
len(r.pages)

31

In [ ]:
#| eval: false
r.pages[2].markdown

In [ ]:
#| eval: false 
print(r.pages[3].markdown)

# 1. Introduction 

In 2016, the EU and IOM launched the EU-IOM Joint Initiative for Migrant Protection and Reintegration, with as 
overall objective "To contribute to facilitating orderly, safe, regular and rights-based migration through the 
facilitation of dignified voluntary return and the implementation of development-focused and sustainable 
reintegration policies and processes". The EU-IOM Joint Initiative in the Horn of Africa (JI-HOA) ${ }^{1}$ 
commenced in March 2017 in the Khartoum Process countries, with a focus on Djibouti, Ethiopia, Somalia, and Sudan. 
${ }^{2}$ The programme was coordinated by a Regional Coordination Unit (RCU) based in the IOM Regional Office for 
the East and Horn of Africa region (Nairobi, Kenya).

In accordance with the programme planning, the JI-HoA underwent a Mid-Term Evaluation in 2019 and a Final 
Independent Evaluation in 2022/2023, covering the 2017-2022 period, conducted by PPMI Group and commissioned by 
IOM. The evaluation covered the four JI core countries in the HoA, namely Ethiopia, Somalia, Sudan and Djibouti, as
well as the RCU based in Kenya given regionally implemented and coordinated activities, looking at all pillars of 
the Action (migration data; capacity building; awareness raising; protection and voluntary assisted return; and 
individual and communitybased reintegration).

The final evaluation assessed the relevance, effectiveness, efficiency, coherence, and sustainability of the 
programme, with an eye on informing future programming. To this end, the evaluation assessed the achievements of 
the programme against its key objectives, including a re-examination of the relevance of the objectives and of the 
design. It also identified factors that have facilitated or impeded the achievement of the objectives.

To gather the necessary data, the evaluation relied on desk research, interviews with stakeholders, focus groups 
with beneficiaries, and a validation workshop. Furthermore, a range of monitoring and evaluation activities were 
already implemented under the JI-HoA. The results of these M\&E activities formed an important data source for the 
current evaluation.

The current Report presents the findings of the evaluation against the evaluation criteria and provides holistic 
conclusions and recommendations to build on the achievements of the JI-HoA and improve future programming.

[^0]
[^0]:    ${ }^{1}$ Formal title: Facility on Sustainable and Dignified Return and Reintegration in Support of the 
Khartoum Process
    ${ }^{2}$ As per the project's Description of the Action, some limited activities mainly involving assisted 
voluntary return and reintegration (AVRR) were also carried out in other Khartoum Process countries (Eritrea as 
conditions allow, Kenya, South Sudan, and Uganda) and from countries along the Southern migration routes (notably 
Malawi, Mozambique, Zambia, Zimbabwe, South Africa and Tanzania)

In [ ]:
#| eval: false
print(r.pages[10].markdown)

According to the RA Monitoring and Satisfaction surveys, 56\% of the surveyed returnees were satisfied or very 
satisfied with the reintegration assistance support provided by the JI-HoA ${ }^{17}$.

Focus Group Discussions highlighted the importance of the economic assistance to enable returnees to develop 
sources of income (e.g. through start-up businesses or employment). Since returnees are coming back usually 
"empty-handed", they are experiencing shame, guilt, and are stigmatised by their communities and relatives. The 
economic support offered by the JI-HoA not only provides them with resources to start their business but also 
restores their dignity and self-trust ${ }^{18}$.

In the Focus Group Discussions (FGDs), some returnees pointed out that the overall value of the economic assistance
was not enough. ${ }^{19}$ This was also confirmed by a stakeholder from Sudan, who suggested that returnees needed
more money to satisfy their livelihood during the time of planting, growing and harvesting, since they put most of 
the microbusiness support into the producing and often have no other sources of income ${ }^{20}$. The 
microbusiness assistance did not always correspond to the knowledge of the recipient or the local context. Namely, 
a few of the returnees revealed that they were not consulted or received different support than they selected, so 
they had no other option, but just to accept the support ${ }^{21}$.

In terms of psychosocial needs, research results presented at the Research and Evidence Facility (REF) conference 
of June 2022 indicated that the incidence of Common Mental Disorders (CMD) is significantly higher among JI-HoA 
beneficiaries in Ethiopia when compared with non-migrant control group ${ }^{22}$. The high prevalence of CMD among
returned migrants from the Middle East was also found in another study ${ }^{23}$. This indicates that irregular 
migrants are much more affected by the CMD compared to people who do not migrate. The specific focus of the JI-HoA 
on MHPSS therefore addresses key difficulties faced by migrants.

However, in the execution, some gaps were found in the correspondence of specific activities to returnees' 
psychosocial needs. The JI-HoA Programme's Lessons Learned from the Psychosocial Support Component Report noted 
gaps in the post-return psychosocial support (e.g., lack of MHPSS service, unclear information about compensation 
of the treatment, high cost, stigma, and low awareness about MHPSS needs among communities) ${ }^{24}$. The 
post-return psychosocial support was not wellintegrated into the main documents of the JI-HoA programme ${ }^{25}$.
This lack of attention led to the lack of

[^0]
[^0]:    ${ }^{17}$ Interim Narrative Report \#4 to the European Union, pg. 59.
    ${ }^{18}$ Kan, Merve. (2022). EU-IOM Joint Initiative Horn of Africa Mental Health and Psychosocial Support 
(MHPSS) Research Report. Pg. 51-52
    ${ }^{19}$ FGD in Somalia
    ${ }^{20}$ Interview with a stakeholder from Sudan
    ${ }^{21}$ Covid-19 Natural Experiment Report, pg. 72, FGD with returnees in Ethiopia
    ${ }^{22}$ IOM RDH. (2022) Partnerships for Migrant Reintegration, pg. 7.
    ${ }^{23}$ Tilahun, M., Workicho, A., \& Angaw, D. A. (2020). Common mental disorders and its associated 
factors and mental health care services for Ethiopian labor migrants returned from Middle East countries in Addis 
Ababa, Ethiopia BMC Health Services Research 20:681 doi: https://doi.org/10.1186/s12913-020-05502-0
    ${ }^{24}$ Kan, Merve. (2022). "Lessons Learned from the Psychosocial Support (PSS) Component of the EU-IOM 
Joint Initiative Programme in Ethiopia, Sudan and Somalia"
    ${ }^{25}$ Mid-Term Review of EU-IOM Joint Initiative for Migrant Protection and Reintegration in the Horn of 
Africa, pg. 15.

In [ ]:
#| eval: false
# from IPython.display import display, Markdown
# Markdown(r.pages[4].markdown)

In [ ]:
# def combine_pages(r):
#     "Combine all pages into single markdown"
#     return "".join([page.markdown + "\n" for page in r.pages])

In [ ]:
# r_all = combine_pages(r); 
# print(r_all);

In [ ]:
#| eval: false
# Extract all lines starting with one or more # using regex
import re
# Find all markdown headings in the text
# ^ - matches start of line
# #+ - matches one or more # characters that denote heading level
# .* - matches any characters until end of line
# $ - matches end of line
# re.MULTILINE flag makes ^ and $ match start/end of each line rather than whole string
re.findall(r'^#+.*$', r_all, re.MULTILINE)

['# **PPMi**',
 '# CONTENTS ',
 '# 1. Introduction ',
 '# 2. Background of the JI-HoA ',
 '### 2.1. Context and design of the JI-HoA',
 '# 2.2. External factors affecting the implementation of the JI ',
 '# 3. Methodology ',
 '# 4. Findings ',
 '### 4.1. Relevance',
 '### 4.1.1. Relevance of programme activities for migrants, returnees, and communities',
 '## Overall performance score for relevance: $3.9 / 5$ <br> Robustness score for the evidence: $4.5 / 5$',
 '### 4.1.1.1 Needs of migrants',
 '### 4.1.1.2 Needs of returnees',
 '# 4.1.1.3 Needs of community members ',
 "### 4.1.2. Programme's relevance to the needs of stakeholders",
 '### 4.1.2.1 Needs of governments',
 '# 4.1.2.2 Needs of other stakeholders ',
 '### 4.2. Coherence',
 "# 4.2.1. The JI-HoA's alignment with the objectives and standards of IOM, and objectives of the EU ",
 '### 4.2.2. Alignment with other initiatives',
 '# 4.3. Effectiveness ',
 '### 4.3.1. Specific Objective 1: Partner countries and relevant stakeholder

In [ ]:
#| export
def save_page_images(
    page, # OCR page object containing images
    dest_folder: Path # Destination folder path
): 
    "Save all images from a page to destination folder as PNG"
    for img in page.images:
        img_data = base64.b64decode(img.image_base64.split(',')[1])
        img_bytes = BytesIO(img_data)
        pil_img = Image.open(img_bytes)
        output_path = dest_folder / img.id
        pil_img.save(output_path)

In [ ]:
#| eval: false
save_page_images(r.pages[4], Path('.'))

In [ ]:
# def clean_fname(fname):
#     "Convert filename to a folder-friendly string"
#     sanitized = fname.replace(' ', '_').replace('-', '_')
#     sanitized = re.sub(r'_{2,}', '_', sanitized) 
#     sanitized = re.sub(r'[^\w\s]', '', sanitized)
#     return sanitized.lower()

## Process Pdfs

In [ ]:
#| exports
def setup_output_dirs(md_library_path="../_data/md_library"):
    "Set up the output directory structure for markdown files"
    md_output_dir = Path(md_library_path)
    mkdir(md_output_dir, parents=True, exist_ok=True, overwrite=False)
    return md_output_dir

In [ ]:
#| exports
def process_report(
    report_path:Path, # Path to the report directory
    md_output_dir:Path # Path to the output directory
    ) -> tuple[list[Path], str]:
    "Process a single report directory containing PDFs"
    pdfs = report_path.ls(file_exts='.pdf')
    eval_report_path = report_path.name
    mkdir(md_output_dir / eval_report_path, parents=True, exist_ok=True, overwrite=False)
    return pdfs, eval_report_path

In [ ]:
#| exports
def process_pdf_page(
    pdf_path:Path, # Path to the PDF file
    page_nb:int, # Page number
    md_output_dir:Path, # Path to the output directory
    eval_report_path:str # Name of the report
    ):
    "Process a single page from a PDF, saving markdown and images"""
    pdf_name = clean_pdf_name(pdf_path.name)
    pdf_dir = md_output_dir / eval_report_path / pdf_name
    mkdir(pdf_dir, parents=True, exist_ok=True, overwrite=False)
    
    r = ocr(pdf_path)
    page = r.pages[page_nb]
    
    # Save markdown
    fname_page = f"page_{page_nb+1}.md"
    page_path = pdf_dir / fname_page
    page_path.write_text(page.markdown)
    
    # Save images
    img_dir = pdf_dir / 'img'
    mkdir(img_dir, parents=True, exist_ok=True, overwrite=False)
    if hasattr(page, 'images') and page.images:
        save_page_images(page, img_dir)
    
    return r, page

In [ ]:
#| exports
def process_pdf(
    pdf_path:Path, # Path to the PDF file
    md_output_dir:Path, # Path to the output directory
    eval_report_path:str # Name of the report
    ):
    "Process all pages in a PDF"
    r = ocr(pdf_path)
    for page_nb in range(len(r.pages)):
        process_pdf_page(pdf_path, page_nb, md_output_dir, eval_report_path)
    return r

In [ ]:
#| exports
def process_all_reports(
    reports:list[Path], # List of paths to the report directories
    md_library_path="../_data/md_library" # Path to the output directory
    ):
    "Process provided evaluation reports and their PDFs"
    md_output_dir = setup_output_dirs(md_library_path)
        
    for report in tqdm(reports, desc="Processing reports"):
        pdfs, eval_report_path = process_report(report, md_output_dir)
        
        for pdf in tqdm(pdfs, desc=f"Processing PDFs in {eval_report_path}", leave=False):
            process_pdf(pdf, md_output_dir, eval_report_path)

In [ ]:
#| eval: false
pdf_library = Path("../_data/pdf_library")
report_id_test = '49d2fba781b6a7c0d94577479636ee6f'
reports = [p for p in pdf_library.ls() if p.name == report_id_test]; reports
process_all_reports(reports)

Processing reports:   0%|          | 0/1 [40:12<?, ?it/s]


KeyboardInterrupt: 